In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [5]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


In [6]:
# Check unique values in Age column
print(train_data['Age'].unique())

# Check if Age column contains NaN or inf values
print(np.isnan(train_data['Age'].values))
print(np.isinf(train_data['Age'].values))
print(train_data['Age'].isnull())
mean_age = train_data['Age'].mean()
train_data['Age'].fillna(mean_age, inplace=True)
mean_testage = test_data['Age'].mean()
test_data['Age'].fillna(mean_testage, inplace=True)
print(train_data['Age'].isnull())

[22.   38.   26.   35.     nan 54.    2.   27.   14.    4.   58.   20.
 39.   55.   31.   34.   15.   28.    8.   19.   40.   66.   42.   21.
 18.    3.    7.   49.   29.   65.   28.5   5.   11.   45.   17.   32.
 16.   25.    0.83 30.   33.   23.   24.   46.   59.   71.   37.   47.
 14.5  70.5  32.5  12.    9.   36.5  51.   55.5  40.5  44.    1.   61.
 56.   50.   36.   45.5  20.5  62.   41.   52.   63.   23.5   0.92 43.
 60.   10.   64.   13.   48.    0.75 53.   57.   80.   70.   24.5   6.
  0.67 30.5   0.42 34.5  74.  ]
[False False False False False  True False False False False False False
 False False False False False  True False  True False False False False
 False False  True False  True  True False  True  True False False False
  True False False False False False  True False False  True  True  True
  True False False False False False False  True False False False False
 False False False False  True  True False False False False False False
 False False False False  True  T

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

y = train_data["Survived"]

features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Embarked"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])





dtc = RandomForestClassifier(random_state=1,max_depth=5)
param_grid = {'n_estimators':[50,100,200]}
grid1 = GridSearchCV(dtc, param_grid, cv=5)
grid1.fit(X,y)
grid1.best_params_


{'n_estimators': 100}

In [8]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=0)
params = {'min_samples_split' : list(range(2, 50)),
          'max_depth' : list(range(2, 11))}

grid = GridSearchCV(estimator=dtc, param_grid=params, cv=5, verbose=10)
grid.fit(X, y)
grid.best_params_

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV 1/5; 1/432] START max_depth=2, min_samples_split=2..........................
[CV 1/5; 1/432] END max_depth=2, min_samples_split=2;, score=0.749 total time=   0.0s
[CV 2/5; 1/432] START max_depth=2, min_samples_split=2..........................
[CV 2/5; 1/432] END max_depth=2, min_samples_split=2;, score=0.792 total time=   0.0s
[CV 3/5; 1/432] START max_depth=2, min_samples_split=2..........................
[CV 3/5; 1/432] END max_depth=2, min_samples_split=2;, score=0.787 total time=   0.0s
[CV 4/5; 1/432] START max_depth=2, min_samples_split=2..........................
[CV 4/5; 1/432] END max_depth=2, min_samples_split=2;, score=0.753 total time=   0.0s
[CV 5/5; 1/432] START max_depth=2, min_samples_split=2..........................
[CV 5/5; 1/432] END max_depth=2, min_samples_split=2;, score=0.787 total time=   0.0s
[CV 1/5; 2/432] START max_depth=2, min_samples_split=3..........................
[CV 1/5; 2/432] END 

{'max_depth': 6, 'min_samples_split': 9}

In [9]:
from sklearn.linear_model import LogisticRegression
logistic_reg = LogisticRegression(multi_class="auto",solver="lbfgs", max_iter=1000, random_state=0)
params = {'C':np.logspace(-4, 4, 20)}
grid = GridSearchCV(estimator=logistic_reg, param_grid=params, n_jobs=5, cv=5, verbose=10)
grid.fit(X, y)
grid.best_params_

Fitting 5 folds for each of 20 candidates, totalling 100 fits


{'C': 0.03359818286283781}

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

rtc = RandomForestClassifier(random_state=1,max_depth=6,n_estimators=100)
logistic_reg = LogisticRegression(multi_class="auto",solver="lbfgs", max_iter=1000, random_state=0, C=0.03359818286283781)
dtc = DecisionTreeClassifier(random_state=1, max_depth=6, min_samples_split=9)
voting_clf = VotingClassifier(estimators=[('lr', logistic_reg),('dtc',dtc) , ('rtc', rtc)], voting='soft')
voting_clf.fit(X, y)
y_test_pred = voting_clf.predict(X_test)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': y_test_pred})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")
print(output)

Your submission was successfully saved!
     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]
